In [85]:
import mysql.connector
import sshtunnel
from mysql.connector import errorcode

def init_mysql():
    global cnx
    cnx = mysql.connector.connect(user='user1', password='tkfkdgody1!', host='52.79.185.101', database='Crawling')
    global cursor
    cursor = cnx.cursor()

In [86]:
add_foreground = ("INSERT IGNORE INTO foreground (title, press, link, day, class) "
                  "VALUES (%(title)s, %(press)s, %(link)s, %(day)s, %(class)s)")
add_background = ("INSERT IGNORE INTO background (tfidf,link) "
                 "VALUES (%(tfidf)s, %(link)s)")
clear_db = ("")

In [87]:
from time import sleep
import requests
from bs4 import BeautifulSoup, NavigableString

def get_html(url):
    _html = ""
    resp = requests.get(url)
    resp.encoding='utf-8'
    if resp.status_code == 200 :
        _html = resp.text
    return _html

In [88]:
#정치 경제 스포츠 연예 사회 국제
#politics economy sports enter national inter
#조선일보 경제는 biz.chosun.com...
CHOSUN = [("http://news.chosun.com/politics",'politics'),("http://news.chosun.com/sports",'sports'), ("http://news.chosun.com/ent",'enter'), ("http://news.chosun.com/national",'national'),( "http://news.chosun.com/international",'inter')]

#동아일보는 패턴 통일
DONGA = [("http://news.donga.com/Politics",'politics'),("http://news.donga.com/Economy",'economy'),("http://news.donga.com/Sports",'sports'),("http://news.donga.com/Enter",'enter'),("http://news.donga.com/Society",'national'),("http://news.donga.com/Inter",'inter')]

#중앙일보 연예가 가요, 방송, 영화로 나눠져있음
JOONGANG = [("http://news.joins.com/politics",'politics'),("http://news.joins.com/money",'economy'),("http://news.joins.com/sports",'sports'),("http://news.joins.com/society",'national'),("http://news.joins.com/world",'inter')]
JOONGANG_CUL = [("http://news.joins.com/culture/song/list",'enter'),("http://news.joins.com/culture/broadcast/list",'enter'),("http://news.joins.com/culture/movie/list",'enter')]

#기사 내용을 담을 변수

num_press = 3
num_class = 6
docs = [[] for i in range(num_class)]

    중앙일보 날짜 가져오기 힘들어서 datetime 패키지를 이용하여 임의로 날짜 입력

In [89]:
from datetime import datetime

In [90]:
def chosun_crawling():
    data_foreground = []
    i = 0
    for clas in CHOSUN:
        html = get_html(clas[0])
        print(clas[0])
        soup = BeautifulSoup(html, 'lxml', from_encoding='utf-8')
        for lt in soup.find_all("dt") :
            try :
                tmp = lt.find('a')
                link = tmp.get('href')
                day = link.split('/')
                day = day[6]+day[7]+day[8]
                text = tmp.get_text()
                data_foreground.append({'title' : text, 'press' : 'chosun',  'link' : link, 'day' : day, 'class' : clas[1]})
                html = get_html(link)
                soup2 = BeautifulSoup(html, 'lxml', from_encoding='utf-8')
                docs[i].append((soup2.find('div',attrs={'class' : 'par'}).get_text(),link))
            except : 
                print('error!!!',link)
        #docs[i].append(tmp_list)
        i = i + 1
        if i == 1 :
            i = i + 1
    cursor.executemany(add_foreground, data_foreground)
    cnx.commit()
    return

def chosun_biz():
    html = get_html('http://biz.chosun.com')
    print('http://biz.chosun.com')
    soup = BeautifulSoup(html, 'lxml', from_encoding='utf-8')
    data_foreground = []
    head = soup.find("div", attrs={'class' : 'mt_art_tit'})
    tmp = head.find('a')
    link = tmp.get('href')
    day = link.split('/')
    day = day[6]+day[7]+day[8]
    text = tmp.get_text()
    data_foreground.append({'title' : text, 'press' : 'chosun',  'link' : link, 'day' : day, 'class' : 'economy'})
    
    html = get_html(link)
    soup2 = BeautifulSoup(html, 'lxml', from_encoding='utf-8')
    docs[1].append((soup2.find('div',id = 'article_2011', attrs={'class':'article'}).get_text(),link))
    
    art = soup.find("div", attrs={'class' : 'mc_art_lst'})
    for lt in art.find_all("li", limit = 30) :
        try :
            tmp = lt.find('a')
            #print(tmp)
            link = tmp.get('href')
            day = link.split('/')
            day = day[6]+day[7]+day[8]
            text = tmp.get_text()
            data_foreground.append({'title' : text, 'press' : 'chosun',  'link' : link, 'day' : day,'class':'economy'})
            html = get_html(link)
            soup2 = BeautifulSoup(html, 'lxml', from_encoding='utf-8')
            docs[1].append((soup2.find('div',id = 'article_2011', attrs={'class':'article'}).get_text(),link))
        except : 
            print('error!!!',link)
    cursor.executemany(add_foreground, data_foreground)
    cnx.commit()
    return 


In [91]:
def donga_crawling():
    data_foreground = []
    #docs = []
    i = 0
    for clas in DONGA:
        html = get_html(clas[0])
        print(clas[0])
        soup = BeautifulSoup(html, 'lxml', from_encoding='utf-8')
        try:
            tmp = soup.find('div',attrs={'class':'articleTop'})
            head = tmp.find("div", attrs={'class' : 'articleMain'})
            #해드라인_main
            head_main = head.find('a')
            link = head_main.get('href')
            day = link.split('/')[6]
            text = head.find(attrs={'class' : 'title'}).get_text()
            data_foreground.append({'title' : text, 'press' : 'donga',  'link' : link, 'day' : day,'class':clas[1]})
            #print(data_foreground)
            html = get_html(link)
            soup2 = BeautifulSoup(html, 'lxml', from_encoding='utf-8')
            docs[i].append((soup2.find('div',attrs={'class':'article_txt'}).get_text(),link))
            for head_sub in tmp.find_all('li') :
                art = head_sub.find('a')
                link = art.get('href')
                day = link.split('/')[6]
                text = art.find(attrs={'class':'title'}).get_text()
                data_foreground.append({'title' : text, 'press' : 'donga',  'link' : link, 'day' : day,'class':clas[1]})
                html = get_html(link)
                soup2 = BeautifulSoup(html, 'lxml', from_encoding='utf-8')
                docs[i].append((soup2.find('div',attrs={'class':'article_txt'}).get_text(),link))

        except :
            for head in tmp.find_all('div',attrs={'class':'artivleMain'}):
                art = head.find('a')
                link = art.get('href')
                day = link.split('/')[6]
                text = art.find(attrs={'class':'title'}).get_text()
                data_foreground.append({'title' : text, 'press' : 'donga',  'link' : link, 'day' : day,'class':clas[1]})
                html = get_html(link)
                soup2 = BeautifulSoup(html, 'lxml', from_encoding='utf-8')
                docs[i].append((soup2.find('div',attrs={'class':'article_txt'}).get_text(),link))

                
            for head in tmp.find_all('div',attrs={'class':'artivleMain02'}):
                art = head.find('a')
                link = art.get('href')
                day = link.split('/')[6]
                text = art.find(attrs={'class':'title'}).get_text()
                data_foreground.append({'title' : text, 'press' : 'donga',  'link' : link, 'day' : day,'class':clas[1]})
                html = get_html(link)
                soup2 = BeautifulSoup(html, 'lxml', from_encoding='utf-8')
                docs[i].append((soup2.find('div',attrs={'class':'article_txt'}).get_text(),link))

        try:
            #issue
            content_issue = soup.find('div',attrs={'class':'issueList'})
            issue = content_issue.find('a', attrs={'class' : 'tit'})
            link = issue.get('href')
            day = link.split('/')[6]
            text = issue.get_text();
            data_foreground.append({'title' : text, 'press' : 'donga',  'link' : link, 'day' : day,'class':clas[1]})
            html = get_html(link)
            soup2 = BeautifulSoup(html, 'lxml', from_encoding='utf-8')
            docs[i].append((soup2.find('div',attrs={'class':'atrticle_txt'}).get_text(),link))
            for issue_sub in content_issue.find_all('li') :
                art = issue_sub.find('a')
                link = art.get('href')
                day = link.split('/')[6]
                text = art.get_text()
                data_foreground.append({'title' : text, 'press' : 'donga',  'link' : link, 'day' : day,'class':clas[1]})
                html = get_html(link)
                soup2 = BeautifulSoup(html, 'lxml', from_encoding='utf-8')
                docs[i].append((soup2.find('div',attrs={'class':'article_txt'}).get_text(),link))
        except :
            print('error!!!',link)
            
        try:
            #최신기사
            contents = soup.find('div',attrs={'class':'articleList_con'})
            for art in contents.find_all('div',attrs={'class':'rightList'}) :
                tmp = art.find('a')
                link = tmp.get('href')
                day = link.split('/')[6]
                text = tmp.find(attrs={'class':'tit'}).get_text()
                data_foreground.append({'title' : text, 'press' : 'donga',  'link' : link, 'day' : day,'class':clas[1]})
                html = get_html(link)
                soup2 = BeautifulSoup(html, 'lxml', from_encoding='utf-8')
                docs[i].append((soup2.find('div',attrs={'class':'article_txt'}).get_text(),link))
        except :
            print('error!!!',link)
        i = i + 1
    cursor.executemany(add_foreground, data_foreground)
    cnx.commit()
    return 

In [92]:
def joongang_crawling():
    day = datetime.today().strftime("%Y%m%d")
    data_foreground = []
    i = 0
    for clas in JOONGANG:
        html = get_html(clas[0])
        print(clas[0])
        soup = BeautifulSoup(html, 'lxml', from_encoding='utf-8')
        origin = soup.find('div',id='content')
        #해드라인
        try:
            art = origin.find('dt')
            tmp = art.find('a')
            link = tmp.get('href')
            text = tmp.get_text()
            data_foreground.append({'title' : text, 'press' : 'joongang',  'link' : link, 'day' : day,'class':clas[1]})
            html2 = get_html(link)
            soup2 = BeautifulSoup(html2, 'lxml', from_encoding='utf-8')
            docs[i].append((soup2.find('div', attrs={'class':'article_body'}, id = 'article_body').get_text(),link))
        except :
            #sport 해드라인
            try:
                for art in origin.find_all('div',attrs={'class':'slide'}):
                    tmp = art.find(attrs={'class':'headline mg'})
                    tmp2 = tmp.find('a')
                    link = tmp2.get('href')
                    text = tmp2.get_text();
                    data_foreground.append({'title' : text, 'press' : 'joongang',  'link' : link, 'day' : day,'class':clas[1]})
                    html2 = get_html(link)
                    soup2 = BeautifulSoup(html2, 'lxml', from_encoding='utf-8')
                    docs[i].append((soup2.find('div', attrs={'class':'article_body'}, id = 'article_body').get_text(),link))
            except:
                print('error!!!',link)
        
        #실시간 주요뉴스
        try:
            art = origin.find('div',attrs={'class':'default_realtime'})
            art = art.find('ul',id='ulItems')
            #print(art)
            for content in art.find_all('li'):
                #print(content)
                tmp = content.find(attrs={'class':'headline mg'})
                tmp2 = tmp.find('a')
                link = 'http://news.joins.com/' + tmp2.get('href')
                text = tmp2.get_text()
                data_foreground.append({'title' : text, 'press' : 'joongang',  'link' : link, 'day' : day,'class':clas[1]})
                html2 = get_html(link)
                soup2 = BeautifulSoup(html2, 'lxml', from_encoding='utf-8')
                docs[i].append((soup2.find('div', attrs={'class':'article_body'}, id = 'article_body').get_text(),link))
        except:
            try:
                art = origin.find('div',attrs={'class':'combination_today'})
                art = art.find('div',attrs={'class':'bd'})
                for content in art.find_all('li'):
                    tmp = content.find(attrs={'class':'headline mg'})
                    tmp2 = tmp.find('a')
                    link = 'http://news.joins.com/' + tmp2.get('href')
                    text = tmp2.get_text()
                    data_foreground.append({'title' : text, 'press' : 'joongang',  'link' : link, 'day' : day,'class':clas[1]})
                    html2 = get_html(link)
                    soup2 = BeautifulSoup(html2, 'lxml', from_encoding='utf-8')
                    docs[i].append((soup2.find('div', attrs={'class':'article_body'}, id = 'article_body').get_text(),link))
            except :
                print('error!!!',link)
        i = i + 1
        if i == 3 :
            i = i + 1
    cursor.executemany(add_foreground, data_foreground)
    cnx.commit()
    return 

def joongang_cul():
    day = datetime.today().strftime("%Y%m%d")
    dosc = []
    #tmp_list = []
    data_foreground = []
    for clas in JOONGANG_CUL:
        html = get_html(clas[0])
        print(clas[0])
        soup = BeautifulSoup(html, 'lxml', from_encoding='utf-8')
        
        origin = soup.find('div',id='content')
        for content in origin.find_all('li'):
            art = content.find(attrs={'class':'headline mg'})
            tmp = art.find('a')
            link = 'http://news.joins.com' + tmp.get('href')
            text = tmp.get_text()
            data_foreground.append({'title' : text, 'press' : 'joongang',  'link' : link, 'day' : day,'class':clas[1]})
            html2 = get_html(link)
            soup2 = BeautifulSoup(html2, 'lxml', from_encoding='utf-8')
            docs[3].append((soup2.find('div', attrs={'class':'article_body'}, id = 'article_body').get_text(),link))
    cursor.executemany(add_foreground, data_foreground)
    cnx.commit()
    return 



    scikit-Learn 패키지를 이용한 TF-IDF 계산


In [93]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import nltk, re, operator
import pandas as pd
from konlpy.tag import Twitter, Kkma

def cal_weight():
    tfidf_list = []
    t = Twitter()
    k = Kkma()
    stopwords = ['은','는','이','가','의','에서','.','\n','\xa0',re.compile('^A-Za-z*$')]
    for i in range(num_class):
    #for i in range(1):
        nouns = []
        for article in docs[i]:
            if article[0] is not '':
                nouns.append(' '.join([noun for noun in t.nouns(str(article[0])) if noun not in stopwords and len(noun) > 1]))        
        vec = TfidfVectorizer(stop_words = stopwords)
        fitted = vec.fit(nouns)
        tfidf_res = fitted.transform(nouns)
        vocab = fitted.get_feature_names()
        j = 0
        for article in tfidf_res.toarray():
            idf = sorted(zip(vocab,article), key=lambda kv:kv[1])[-3:]
            tmp = idf[0][0]+' '+idf[1][0] + ' ' + idf[2][0]
            tfidf_list.append({'tfidf':tmp,'link':docs[i][j][1]})
            j = j + 1
    cursor.executemany(add_background, tfidf_list)
    cnx.commit()
    cursor.execute('delete from foreground where link not in (select link from background);')
    cnx.commit()
    print('cal_weight done!')
    return tfidf

In [94]:
from sklearn.manifold import TSNE
from future.utils import iteritems

def embedding(weight):
    print(weight.toarray().shape)
    return

In [99]:
embedding(weight)

(482, 3542)


In [96]:
def main():
    
    init_mysql()
    
    joongang_crawling()
    joongang_cul()
    chosun_crawling()
    chosun_biz()
    donga_crawling();
    
    ret = cal_weight()
    
    return ret

In [97]:
import time

if __name__ == '__main__':
    startTime = time.time()
    weight = main()
    endTime = time.time()
    print(endTime - startTime)
    
cursor.close()
cnx.close()

http://news.joins.com/politics


C:\Anaconda\lib\site-packages\bs4\__init__.py:146: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


http://news.joins.com/money
http://news.joins.com/sports
http://news.joins.com/society
http://news.joins.com/world
http://news.joins.com/culture/song/list
http://news.joins.com/culture/broadcast/list
http://news.joins.com/culture/movie/list
http://news.chosun.com/politics
error!!! http://forum.chosun.com/message/messageView.forum?bbs_id=1010&message_id=1381824
error!!! http://forum.chosun.com/message/messageView.forum?bbs_id=1010&message_id=1381827
http://news.chosun.com/sports
http://news.chosun.com/ent
http://news.chosun.com/national
error!!! http://news.chosun.com/site/data/html_dir/2018/10/04/2018100400403.html
error!!! http://news.chosun.com/site/data/html_dir/2018/10/04/2018100400403.html
error!!! http://news.chosun.com/site/data/html_dir/2018/10/04/2018100400403.html
error!!! http://news.chosun.com/site/data/html_dir/2018/10/04/2018100400403.html
error!!! http://forum.chosun.com/message/messageView.forum?bbs_id=1040&message_id=1381728
error!!! http://forum.chosun.com/message/mes

C:\Anaconda\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


cal_weight done!
177.8609001636505


In [98]:
#weight = cal_weight()